In [17]:
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse

# --- Conexão com o banco ---
usuario = "root"
senha_original = "Sen3180326#*#"
senha = urllib.parse.quote_plus(senha_original)
host = "localhost"
porta = 3306
banco = "labstag"

engine = create_engine(f"mysql+pymysql://{usuario}:{senha}@{host}:{porta}/{banco}")

# --- Funções de consulta ---
def buscar_versiculo(engine, versao, livro, capitulo, versiculo):
    tabela = f"versao_{versao.lower()}"
    query = f"""
    SELECT * FROM {tabela}
    WHERE LOWER(livro) = %s AND capitulo = %s AND versiculo = %s
    """
    df = pd.read_sql_query(query, engine, params=(livro.lower(), capitulo, versiculo))
    if df.empty:
        print("Versículo não encontrado.")
    else:
        row = df.iloc[0]
        print(f"\n{row['livro']} {row['capitulo']}:{row['versiculo']} ({row['versao'].upper()})")
        print(row["texto"])

        # Exporta para Excel
        nome_arquivo = f"{versao}_{livro}_{capitulo}_{versiculo}.xlsx".replace(" ", "_")
        df.to_excel(nome_arquivo, index=False)
        print(f"\n📄 Versículo exportado para o arquivo: {nome_arquivo}")

def buscar_capitulo(engine, versao, livro, capitulo):
    tabela = f"versao_{versao.lower()}"
    query = f"""
    SELECT * FROM {tabela}
    WHERE LOWER(livro) = %s AND capitulo = %s
    ORDER BY versiculo
    """
    df = pd.read_sql_query(query, engine, params=(livro.lower(), capitulo))
    if df.empty:
        print("Capítulo não encontrado.")
    else:
        print(f"\n{livro} {capitulo} ({versao.upper()})\n")
        for _, row in df.iterrows():
            print(f"{row['versiculo']}: {row['texto']}")

        # Exporta para Excel
        nome_arquivo = f"{versao}_{livro}_{capitulo}.xlsx".replace(" ", "_")
        df.to_excel(nome_arquivo, index=False)
        print(f"\n📄 Capítulo exportado para o arquivo: {nome_arquivo}")

def listar_livros(engine, versao):
    tabela = f"versao_{versao.lower()}"
    query = f"""
    SELECT DISTINCT livro, abrev FROM {tabela}
    ORDER BY livro
    """
    df = pd.read_sql_query(query, engine)
    if df.empty:
        print("Nenhum livro encontrado.")
    else:
        print(f"\nLivros da versão {versao.upper()}:\n")
        print(df.to_string(index=False))

# --- Menu Interativo ---
def menu():
    while True:
        print("\n--- Menu da Bíblia ---")
        print("1. Listar livros")
        print("2. Buscar versículo")
        print("3. Buscar capítulo")
        print("0. Sair")
        
        opcao = input("Escolha uma opção: ")

        if opcao == "0":
            print("Saindo...")
            break

        versao = input("Versão da Bíblia (aa, acf, nvi): ").strip().lower()
        if versao not in ["aa", "acf", "nvi"]:
            print("Versão inválida. Tente novamente.")
            continue

        if opcao == "1":
            listar_livros(engine, versao)

        elif opcao == "2":
            livro = input("Livro: ").strip()
            capitulo = int(input("Capítulo: "))
            versiculo = int(input("Versículo: "))
            buscar_versiculo(engine, versao, livro, capitulo, versiculo)

        elif opcao == "3":
            livro = input("Livro: ").strip()
            capitulo = int(input("Capítulo: "))
            buscar_capitulo(engine, versao, livro, capitulo)

        else:
            print("Opção inválida. Tente novamente.")

# Rodar menu
if __name__ == "__main__":
    menu()



--- Menu da Bíblia ---
1. Listar livros
2. Buscar versículo
3. Buscar capítulo
0. Sair


Escolha uma opção:  2
Versão da Bíblia (aa, acf, nvi):  aa
Livro:  Judas
Capítulo:  1
Versículo:  21



Judas 1:21 (AA)
conservai-vos no amor de Deus, esperando a misericórdia de nosso Senhor Jesus Cristo para a vida eterna.

📄 Versículo exportado para o arquivo: aa_Judas_1_21.xlsx

--- Menu da Bíblia ---
1. Listar livros
2. Buscar versículo
3. Buscar capítulo
0. Sair


Escolha uma opção:  0


Saindo...
